In [10]:
import pandas as pd
from dataAug.tools import dataAugForTestSMOTE, dataAugForTestBorderlineSMOTE
from processTools import processRealData
import torch
from ModelClassify_4 import ModelClassify
from targeTools import testThresholdFive, Accuracy

In [11]:
feature_pd = pd.read_csv('/home/kongge/projects/new_protT5/data/AAC_DPC_T5_578.csv')
labels_pd = pd.read_csv("/home/kongge/projects/new_protT5/data/mutil_label_578.csv")

In [12]:
feature_pd

,0,1,2,3,4,5,6,7,8,9,...,1434,1435,1436,1437,1438,1439,1440,1441,1442,1443
0,0.038674,0.005525,0.027624,0.066298,0.060773,0.077348,0.000000,0.099448,0.044199,0.154696,...,0.036818,0.026898,0.010529,-0.087857,0.008474,0.021624,0.037671,-0.042609,-0.033987,0.001863
1,0.097744,0.015038,0.067669,0.090226,0.000000,0.060150,0.007519,0.060150,0.052632,0.150376,...,0.050693,0.058083,0.040556,-0.099527,-0.021974,0.034129,-0.015654,-0.016042,0.018142,0.009868
2,0.078212,0.000000,0.039106,0.100559,0.044693,0.050279,0.005587,0.100559,0.089385,0.156425,...,-0.017891,0.025206,0.068204,-0.120173,-0.041826,-0.001013,-0.015208,0.003502,0.038556,0.032665
3,0.058824,0.005348,0.037433,0.080214,0.048128,0.016043,0.010695,0.101604,0.085561,0.144385,...,-0.005363,0.065209,0.039654,-0.100884,0.034206,0.002743,-0.040962,0.035129,0.013996,0.079778
4,0.135870,0.000000,0.059783,0.086957,0.027174,0.038043,0.010870,0.054348,0.048913,0.119565,...,-0.019232,-0.013705,0.001783,-0.069890,0.016761,0.059995,-0.020010,-0.052366,-0.008041,-0.017827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,0.063158,0.010526,0.042105,0.068421,0.047368,0.031579,0.010526,0.052632,0.047368,0.089474,...,-0.082709,-0.038805,0.049082,-0.072492,0.061679,0.055330,-0.054334,0.022155,0.081683,0.008963
574,0.051867,0.006224,0.056017,0.093361,0.045643,0.064315,0.004149,0.066390,0.064315,0.132780,...,0.005489,-0.021214,-0.028315,-0.027093,0.051935,0.024115,-0.000847,0.049707,-0.002306,0.000640
575,0.045662,0.013699,0.027397,0.027397,0.063927,0.091324,0.027397,0.095890,0.027397,0.141553,...,-0.021107,0.007253,0.054399,-0.098169,0.005296,0.016531,0.015466,-0.041228,0.019770,-0.002437
576,0.095070,0.024648,0.028169,0.028169,0.077465,0.052817,0.024648,0.066901,0.038732,0.102113,...,-0.052751,-0.001265,0.058449,-0.104947,0.083710,0.017370,-0.021055,0.059610,0.035077,0.055231


In [1]:
smote_multiple = {}
for item in range(1, 5, 2):
    G_feature, G_label = dataAugForTestSMOTE(feature_pd, labels_pd, item, 1444)
    R_feature, R_label = processRealData(feature_pd, labels_pd, feature_num=1444)

    train_feature = pd.concat([R_feature, G_feature], axis=0)
    train_label = pd.concat([R_label, G_label], axis=0)

    test_df = pd.read_csv("/home/kongge/projects/new_protT5/data/AAC_DPC_T5_test_mutil_label_122_finally.csv")

    from torch.utils.data import TensorDataset, DataLoader
    datasetTrain = TensorDataset(torch.tensor(train_feature.values), torch.tensor(train_label.values))
    batch_size = len(train_feature)
    shuffle = True
    dataloaderTrain = DataLoader(datasetTrain, batch_size=batch_size, shuffle=True)

    datasetTest = TensorDataset(torch.tensor(test_df.iloc[:,:1444].values), torch.tensor(test_df.iloc[:,-4:].values))
    # 创建 DataLoader 对象，设置批量大小和是否打乱数据
    batch_size = len(datasetTest)
    dataloaderTest = DataLoader(datasetTest, batch_size=batch_size, shuffle=False)

    model = ModelClassify()
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


    import warnings
    warnings.filterwarnings("ignore")
    GMList = {}
    for epoch in range(200):
        model.train()
        total_loss = 0.0
        for idx, data in enumerate(dataloaderTrain, 0):
            inputs, labels = data
            labels = labels.float()
            inputs = inputs.float()
            out = model(inputs)
            loss = criterion(out, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloaderTrain)
        threshold = 0.5
        labels_cov = torch.where(out > threshold, torch.tensor(1), torch.tensor(0))
        print(f"Epoch [{epoch+1}/{300}], Average Loss: {avg_loss:.4f}, ACC: {Accuracy(labels.int(), labels_cov)}")
        GMScore1 = testThresholdFive(epoch, model, dataloaderTest)
        GMList[epoch] = GMScore1

    best_key = max(GMList, key=GMList.get)
    best_value = GMList[best_key]
    smote_multiple[item] = best_value

NameError: name 'dataAugForTestSMOTE' is not defined

In [ ]:
smote_multiple